<a href="https://colab.research.google.com/github/mar1po5-a/ezen/blob/main/ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain-google-genai
!pip install -q langchain langchain-community chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.1 MB/s eta 0:0

In [ ]:
import os
from google.colab import userdata

In [ ]:
# Gemini API 키 비밀관리 설정 후 진행
# userdata.get() -> 저장된 비밀 값을 안전하게 불러옴
try:
  os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except userdata.SecretNotFoundError:
  print("ERROR: GOOGLE_API_KEY not found in Colab secrets. Pleas set it.")

In [ ]:
# 임베딩 모델
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# 벡터 DB
from langchain_community.vectorstores import Chroma
# 문서 표현
from langchain_core.documents import Document
# 텍스트 크기 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 프롬프트 생성
from langchain_core.prompts import ChatPromptTemplate
# 체인 내에서 데이터를 그대로 전달
from langchain_core.runnables import RunnablePassthrough
# LLM 출력을 문자열로 변환
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# 샘플 데이터
documents_data = [
    "대한민국의 수도는 서울입니다. 서울은 역사와 현대가 공존하는 도시입니다.",
    "제주도는 아름다운 자연 경관으로 유명한 대한민국의 섬입니다. 한라산 국립공원이 대표적입니다.",
    "부산은 대한민국 제2의 도시이자 최대 항구 도시입니다. 해운대 해수욕장이 유명합니다.",
    "인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력 등을 컴퓨터 프로그램을 통해 구현하는 기술입니다.",
    "머신러닝은 인공지능의 한 분야로, 컴퓨터가 데이터를 통해 스스로 학습하고 예측하는 기술입니다."
    "1. 청년내일채움공제 \
[지원조건]\
중소기업에 정규직으로 취업한 청년에 해당하는 사람 [수령금액] 1,500만 원을 수령하게 됩니다."
]

In [ ]:
# 샘플 데이터를 Document 객체로 변환
# page_context와 metadata를 가짐
docs = [Document(page_content=text, metadata={"source": f"doc_{i+1}"})
for i, text in enumerate(documents_data)]

print(docs)

[Document(metadata={'source': 'doc_1'}, page_content='대한민국의 수도는 서울입니다. 서울은 역사와 현대가 공존하는 도시입니다.'), Document(metadata={'source': 'doc_2'}, page_content='제주도는 아름다운 자연 경관으로 유명한 대한민국의 섬입니다. 한라산 국립공원이 대표적입니다.'), Document(metadata={'source': 'doc_3'}, page_content='부산은 대한민국 제2의 도시이자 최대 항구 도시입니다. 해운대 해수욕장이 유명합니다.'), Document(metadata={'source': 'doc_4'}, page_content='인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력 등을 컴퓨터 프로그램을 통해 구현하는 기술입니다.'), Document(metadata={'source': 'doc_5'}, page_content='머신러닝은 인공지능의 한 분야로, 컴퓨터가 데이터를 통해 스스로 학습하고 예측하는 기술입니다.1. 청년내일채움공제 [지원조건]중소기업에 정규직으로 취업한 청년에 해당하는 사람 [수령금액] 1,500만 원을 수령하게 됩니다.')]


In [ ]:
# 검색 정확도를 높이고, 토큰을 효율적으로 활용하기 위해 chunk 단위로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200, # 각 청크의 크기
    chunk_overlap = 20 # 청크 간 중복되는 글자 수
)
# split_documents -> 그냥 변수명
# .split_documents() -> method
split_documents = text_splitter.split_documents(docs)

print(f"원본 문서 개수: {len(docs)}")
print(f"분할된 청크 개수: {len(split_documents)}")
# print("\n분할된 첫 번째 청크 예시:")
print(split_documents[0])

원본 문서 개수: 5
분할된 청크 개수: 5
page_content='대한민국의 수도는 서울입니다. 서울은 역사와 현대가 공존하는 도시입니다.' metadata={'source': 'doc_1'}


In [ ]:
# 임베딩 모델 초기화
# models/text_embedding-004 -> 최신 고성능 임베딩 모델
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# llm 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0.3 # 창의성
    # 시스템 프롬프트를 사용자 프롬프트 형태로 변환하여 전달
    # API 호환성과 model의 특성에 따라 유동적으로 선택하는 옵션
    # convert_system_message_to_human=True
)

print("Gemini 임베딩 모델 및 LLM이 성공적으로 초기화되었습니다.")

Gemini 임베딩 모델 및 LLM이 성공적으로 초기화되었습니다.


In [ ]:
# 벡터 저장소 생성
# Chorma.from_documents()는 모든 작업을 한 번에 수행
# 1. 문서 청크를 임베딩 모델을 사용해 벡터로 변환
# 2. 벡터와 원본 텍스트, 메타 데이터를 ChromaDB에 저장
# persist_directory를 지정하면 해당 경로에 DB 파일이 저장됨. -> 재사용 가능
vectorstore_path = "./chroma_db_rag_store"
vectorstore = Chroma.from_documents(
    documents=split_documents, # 분할된 문서
    embedding=embeddings, # 임베딩 모델
    # persist : 지속하다
    persist_directory=vectorstore_path # DB 저장할 디렉토리
)

# 저장된 DB를 다시 로드할 때 사용 (재생성 x)
# vectorstore = Chroma(persist_directory=vectorstore_path, embedding_function=embeddings)

print(f"문서들이 ChromaDB에 성공적으로 임베딩 및 저장되었습니다. (저장 경로: {vectorstore_path})")
# 기존의 RDB : table / Vector DB : collection
print(f"ChromaDB에 저장된 총 벡터(청크) 수: {vectorstore._collection.count()}")

문서들이 ChromaDB에 성공적으로 임베딩 및 저장되었습니다. (저장 경로: ./chroma_db_rag_store)
ChromaDB에 저장된 총 벡터(청크) 수: 10


In [ ]:
# 벡터 저장소에서 유사도 높은 문서를 검색하는 검색기 생성
# retriever : 되찾다, 회수하다
# k=3은 가장 유사한 문서 3개를 가져오도록 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 검색기 테스트
query_test = "대한민국의 수도는 어디인가요?"
retrieved_docs_test = retriever.invoke(query_test)
print(f"'{query_test}'에 대한 검색 결과 (상위 {len(retrieved_docs_test)}개)")
for i, doc in enumerate(retrieved_docs_test):
  # 메타데이터 중 source를 가져오되, 존재하지 않으면 기본 값을 가져옴
  print(f" {i+1}, {doc.page_content} (출처: {doc.metadata.get('source', 'N/A')})")

'대한민국의 수도는 어디인가요?'에 대한 검색 결과 (상위 3개)
 1, 대한민국의 수도는 서울입니다. 서울은 역사와 현대가 공존하는 도시입니다. (출처: doc_1)
 2, 대한민국의 수도는 서울입니다. 서울은 역사와 현대가 공존하는 도시입니다. (출처: doc_1)
 3, 제주도는 아름다운 자연 경관으로 유명한 대한민국의 섬입니다. 한라산 국립공원이 대표적입니다. (출처: doc_2)


In [ ]:
# RAG 파이프라인 구성 : 정보 검색 - 문맥 정보 준비 - 프롬프트 구성
# - 답변 생성 및 출력 일련의 과정이 파이프라인

# 프롬프트 템플릿 정의
# {context} : 검색된 문서 내용, {question} : 사용자 질
prompt_template = """당신은 질문에 친절하고 상세하게 답변하는 AI 어시스턴트입니다.
주어진 문맥(context) 정보를 바탕으로 질문에 답변해주세요. 문맥에서 답을 찾을 수 없다면, "제공된 정보만으로는 답변하기 어렵습니다."라고 솔직하게 말해주세요.

문맥:
{context}

질문:
{question}

답변:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

# RAG 체인 정의
# retriever로 문서를 검색하고
# 검색된 문서들의 page_content를 하나의 문자열로 합
# LCEL 사용 : |(파이프 기호)의 사용, 파이프라인 또는 체인을 더 쉽고 유연하게 구성하는 방식
rag_chain = (
    {"context": retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)),
    # RunnablePassthrough.assian() : 체인 내에서 새로운 키를 생성하고 값을 할당
     # question에 초기 입력을 그대로 할당하기 위해 키:값으로 세팅
     "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser() # llm 출력을 일반 문자열로 변환
)

print("RAG 체인이 성공적으로 구성되었습니다.")


RAG 체인이 성공적으로 구성되었습니다.


In [ ]:
# RAG 체인에 질문을 입력하여 답변 생성

question1 = "대한민국의 수도는 어디이고, 그 도시는 어떤 특징이 있나요?"
print(f"\n[질문 1]: {question1}")
answer1 = rag_chain.invoke(question1)
print(f"[답변 1]: {answer1}")

question2 = "AI 기술에 대해 설명해주세요."
print(f"\n[질문 2]: {question2}")
answer2 = rag_chain.invoke(question2)
print(f"[답변 2]: {answer2}")

question3 = "부산의 유명한 음식은 무엇인가요?" # 샘플 문서에 없는 내용
print(f"\n[질문 3]: {question3}")
answer3 = rag_chain.invoke(question3)
print(f"[답변 3]: {answer3}")

question4 = "청년내일채움공제 지원조건 " # 샘플 문서에 없는 내용
print(f"\n[질문 4]: {question4}")
answer4 = rag_chain.invoke(question4)
print(f"[답변 4]: {answer4}")


[질문 1]: 대한민국의 수도는 어디이고, 그 도시는 어떤 특징이 있나요?
[답변 1]: 대한민국의 수도는 서울입니다.  서울은 역사와 현대가 공존하는 도시라는 특징이 있습니다.

[질문 2]: AI 기술에 대해 설명해주세요.
[답변 2]: AI 기술은 인간의 학습능력, 추론능력, 그리고 지각능력을 컴퓨터 프로그램을 통해 구현하는 기술입니다.  즉, 인간이 하는 것처럼 배우고, 추론하고, 세상을 인지하는 능력을 컴퓨터 시스템에 부여하는 것을 목표로 합니다.

[질문 3]: 부산의 유명한 음식은 무엇인가요?
[답변 3]: 제공된 정보만으로는 답변하기 어렵습니다.  제공된 문맥에는 서울과 제주도에 대한 정보만 있고, 부산에 대한 정보는 없습니다.

[질문 4]: 청년내일채움공제 지원조건 
[답변 4]: 제공된 정보만으로는 답변하기 어렵습니다.  제공된 문맥은 대한민국의 수도와 제주도에 대한 정보만 담고 있고, 청년내일채움공제 지원조건에 대한 내용은 포함되어 있지 않습니다.


In [ ]:
# ChromaDB 파일 정리
# 해당 코드 실행 시 vectorstore_path에 저장된 DB 데이터가 삭제됨

import shutil

try:
  # vectorstore 객체가 존재하면 컬렉션 먼저 삭제 시도
  if 'vectorstore' in globals() and vectorstore is not None:
    print(f"'{vectorstore_path}' 디렉토리를 삭제하여 ChromaDB 데이터를 정리합니다.")
    shutil.rmtree(vectorstore_path)
    print("ChromaDB 데이터 정리가 완료되었습니다.")
except FileNotFoundError:
    print(f"'{vectorstore_path}' 디렉토리가 이미 존재하지 않습니다.")
except Exception as e:
    print(f"ChromaDB 데이터 정리 중 오류 발생: {e}")

'./chroma_db_rag_store' 디렉토리를 삭제하여 ChromaDB 데이터를 정리합니다.
ChromaDB 데이터 정리가 완료되었습니다.
